<a href="https://colab.research.google.com/github/kadirov1194/Building_energy_forecasting/blob/main/Building_energy_consumption_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
Predicting energy budget of a building for a month
 - Weather data for 3 years
 - Energy consumption record of 3 years
 - Energy cost of a month

In [ ]:
# we will use the pandas library for data analysis and manipulation.
import pandas as pd

# Often we need some functions from numpy for adding support for large, multi-dimensional arrays and matrices.
import numpy as np

# For data visualization, we import matplotlib
import matplotlib.pyplot as plt
     

Load the building electricity consumption data

In [ ]:
from google.colab import drive  # Mount to Google Colab/ MyDrive/ Colab
drive.mount('/content/drive')

In [ ]:
Energy1 = pd.read_excel('/content/drive/MyDrive/demand_response/data/Building energy consumption racord.xlsx')
Energy1

In [ ]:
# set time column as index
Energy = Energy1.set_index('Time')
Energy

In [ ]:
# Check the description of the data
Energy.describe()

In [ ]:
plt.boxplot(Energy['building 41']) # Energy.loc[:,'building 12'] or Energy.iloc[:,0]
plt.show()

In [ ]:
# a visual representation of the data description
import seaborn as sns
sns.boxplot(Energy['building 41'])

In [ ]:
# Load the weather data from the excel fiie
Path = "/content/drive/MyDrive/demand_response/data/WeatherData.xlsx"
knmi = pd.read_excel(Path)
knmi

In [ ]:
#Set the Time column as index
knmi = knmi.set_index("Time")
knmi

In [ ]:
#concatenating the datasets of weather data and electricity consumption
df = pd.concat([knmi, Energy], axis=1) #axis =1 for considering the columns
df

In [ ]:
# check missing data status
df.isna().sum()

In [ ]:
# cross check the availability of the data with missingno library
import missingno as msno
msno.bar(df)

We have no missing values in the dataframe. Our output (dependent variable) is building 12 column. other columns of the data set are independent values. We can find strong and weak correlation with different variables. With .corr() method, we can utilize Pearson's correlation coefficient which is a measure of the strength of a linear association between two variables.

In [ ]:
plt.figure(figsize = (16,6)) # Create matplotlib figure
sns.heatmap(df.corr(), annot = True, linewidths=1, fmt=".2g", cmap= 'coolwarm') 
# fmt = .1e (scientific notation), .2f (2 decimal places), .3g(3 significant figures), .2%(percentage with 2 decimal places)
plt.xticks(rotation='horizontal')

From the heatmap, we see temperature (Temp) correlates very positively with building electricity demand. Relative humidity (U) and hourly sum of precipitation (RH) are two highest negatively correlated features. in addition, both of these features are also multi-collinear. Which means, either of them can be utilized for predicting electricity demand.


# Plot energy consumption data againts U and Temp

In [ ]:
# Resample the energy of the building over a week using the resmaple function and the mean  function. 
df_sum_weekly = df['building 41'].resample('W').mean()

# Resample the temperature over a week. 
df_feature1= df["Temp"].resample("W").mean()

# Resample the relative humidity over a week. 
df_feature2 = df["U"].resample("W").mean()

In [ ]:
# plot the result
fig,ax = plt.subplots(figsize=(24,8))  # Create matplotlib figure
ax.plot(df_sum_weekly.index, df_sum_weekly, color="red",marker="o")
ax.set_ylabel("KWh")
ax.set_xlabel('Date')
ax2 = ax.twinx() #Create a new Axes with an invisible x-axis and an independent y-axis positioned opposite to the original one (i.e. at right).
ax3 = ax.twinx()
ax2.plot(df_sum_weekly.index, df_feature1, color="blue", marker="o")
ax2.set_ylabel("°C")
ax3.plot(df_sum_weekly.index, df_feature2, color="green", marker="o")
ax3.set_ylabel("grams/cubic meter")
ax3.spines["right"].set_position(("axes", 1.05))
fig.legend(["Resampled Weekly Energy Consumption","Resampled Weekly Temperature","Resampled Weekly Relative Humidity"], loc='upper right')
fig.show()

We see that energy demand of a building varies with temperature. Variations of the energy consumption across various seasons are also visible. Negative linear correlation of Relative Humidity can be explained. It is not just correlational with Energy consumption but also has high negative correlation (-0.57) with temperature. The correlations observed are well expected.

# Feature selection
We can now select features based on their strong coorealtion with the output and remove some input features which are strongly coorelated with each other to avoid the problem of multicolinearity. It is a phenomenon in which one predictor variable in a multiple regression model can be linearly predicted from the others with a substantial degree of accuracy.
# Exploring non-linear correlation between Energy with Hour and Month
We use spearman's correlation between two variables. The Spearman rank-order correlation coefficient is a nonparametric measure of the monotonicity of the relationship between two datasets. Unlike the Pearson correlation, the Spearman correlation does not assume that both datasets are normally distributed. This one varies between -1 and +1 with 0 implying no correlation. Correlations of -1 or +1 imply an exact monotonic relationship. Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, y decreases.

In [ ]:
# calculate the spearmans's correlation between two variables
from scipy.stats import spearmanr

#filter columns from dataframe
energy = np.array(df["building 41"]) 
hour = np.array(df["HH"])
month= np.array(df["month"])

# calculate spearman's correlation
corr1, _ = spearmanr(energy, hour)
corr2,_ = spearmanr(energy, month)
print('Spearmans correlation between Energy and hour feature: %.3f' % corr1)
print('Spearmans correlation between Energy and month feature: %.3f' % corr2)

We see, the energy consumption has a seasonal effet which is reflected on the different months of the year. So it has more correlation with month than hours of the day.

In [ ]:
#Reduce number of features with lower correlation values or it has an inverse effect on the results of the model.
knmi_updated= knmi.loc[:, ~knmi.columns.isin(["TD","U","DR","FX"])] # ~ sign drops the columns we select
knmi_updated

# Now we develop a machine learning regression model based on the weather parameters to predict the energy consumption of the building.
Various forecasting techniques can be utilized with machine learning models. (Deng et al., 2018) tested the performance of various machine learning models on one of the largest database on buildings in CBECS, and found both Support Vector Machine (SVM) and Random Forest (RF) being able to handle the non-linear relationships better as they perform dynamic local investigations better rather than global optimization. Therefore, we are going to consider SVM and RF to develop the model.

In [ ]:
#Splitting the data into training (80%) and testing (20%) set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(knmi_updated, Energy, test_size = 0.2, random_state = 0)
y_train

In [ ]:
y_train = y_train.values.ravel() #ravel is a numpy function to change a 2-dimensional array or a multi-dimensional array into a continuous flattened array.
y_train

In [ ]:
y_test = y_test.values.ravel()
y_test

In [ ]:
# importing regression model 
from sklearn.svm import SVR

#Creating an instance or object of the support vector machine regressor class
SVReg = SVR(kernel= 'rbf') # It must be one of 'linear', 'poly', 'rbf', 'sigmoid' (rbf - Radial Basis Function is used in machine learning to find a non-linear regression line.)

# fitting the regression model to the training dataset
SVReg.fit(X_train, y_train) #Fit the SVM model according to the given training data.

Three hyperparameters for the SVR function are Epsilon, C and Gamma.

Epsilon represents the maximum error allowed in the function, and determines the desired accuracy of the model. A smaller ϵ indicates that a more accurate model is required.

Parameters C is used to set the tolerance for points which fall outside of the error boundaries set by ϵ. A larger value for C indicates a larger tolerance for points outside of ϵ.

The gamma parameter defines the influence of a single point on the model. A low value indicates that points have a far reach, meaning that also points that are situated far away from the regression line are taken into account. A low value results in a more linear regression line. On the other hand, a high value for gamma indicates that points have a close reach, which results in a more complex or wobbly line around the nearby points. With default 'scale', the value of gamma is 1 / (n_features * X.var()).

In [ ]:
# predicting on the training data
Predicted_Train= SVReg.predict(X_train)
Predicted_Train

In [ ]:
# To evaluate the performance of the model, importing error metrics function
from sklearn.metrics import r2_score #(coefficient of determination) regression score function.
from sklearn.metrics import mean_squared_error #The MSE indicates the average distance of the best fit regression line to the observed values.

print(r2_score(y_train,Predicted_Train))
print(mean_squared_error(y_train,Predicted_Train))

# Scaling to improve the model performance
Scaling is used to bring all features to the same level of magnitudes. Without scaling, the features with high magnitudes will have more weight in the ‘best fit’ calculation, which tries to minimize the distance between the fit line and the observed values (Asaithambi, 2017).

In [ ]:
# Import the required packages
from sklearn.preprocessing import StandardScaler #standardizes the data to a range in which the mean is equal to 0 and the standard deviation is 1. It assumes the data is normally distributed.
from sklearn.preprocessing import MinMaxScaler #normalizes the data and brings the values between 0 (lowest value) and 1 (highest value)
from sklearn.preprocessing import RobustScaler #standardizes the data. But is more robust to outliers because it only scales the data according to the Interquartile Range (IQR) between the 1st and 3rd quartile.

#Generate the scaler
sc1= StandardScaler()
sc2= MinMaxScaler()
sc3= RobustScaler()

In [ ]:
#Scaling the input data
X1 = sc1.fit_transform(knmi_updated)
X2 = sc2.fit_transform(knmi_updated)
X3 = sc3.fit_transform(knmi_updated)

#We do not need to scale the output data as we have only one output.

In [ ]:
#plotting to visually explore the scaled features
sns.distplot(X1,color="red",label="StdScaler")
sns.distplot(X2,color="blue",label="MinMax")
sns.distplot(X3,color="black",label="RobustScaler")
plt.legend()

In [ ]:
#Split your data set into training (80%) and test data (20%)
X_train, X_test, y_train, y_test = train_test_split(X1, Energy, test_size=0.2, random_state=0, shuffle= "False")
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [ ]:
#building the regressor and fit the training data to the regressor
regr = SVR(kernel='rbf')
regr= regr.fit(X_train, y_train)
regr

In [ ]:
# fitting the regression model to the training data
regr.fit(X_train, y_train) #Fit the SVM model according to the given training data.
# predicting on the training data
predict_train= regr.predict(X_train)

In [ ]:
#testing the model training accuracy 
print(r2_score(y_train, predict_train))
print(mean_squared_error(y_train, predict_train))

In [ ]:
#Predicting on the test data
pred= regr.predict(X_test)
##testing the models accuracy on the test data
print(r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))

Now we test for MnMax scaling

In [ ]:
#Split your data set into training (80%) and test data (20%)
X_train, X_test, y_train, y_test = train_test_split(X2, Energy, test_size=0.2, random_state=0, shuffle= "False")
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

#building the regressor and fit the training data to the regressor
regr = SVR(kernel='rbf')
regr= regr.fit(X_train, y_train)

# fitting the regression model to the training data
regr.fit(X_train, y_train) #Fit the SVM model according to the given training data.
# predicting on the training data
predict_train= regr.predict(X_train)

#testing the model training accuracy 
print(r2_score(y_train, predict_train))
print(mean_squared_error(y_train, predict_train))

In [ ]:
#Predicting on the test data
pred= regr.predict(X_test)
##testing the models accuracy on the test data
print(r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))

Finnaly, we test for Robust scaling

In [ ]:
#Split your data set into training (80%) and test data (20%)
X_train, X_test, y_train, y_test = train_test_split(X3, Energy, test_size=0.2, random_state=0, shuffle= "False")
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

#building the regressor and fit the training data to the regressor
regr = SVR(kernel='rbf')
regr= regr.fit(X_train, y_train)

# fitting the regression model to the training data
regr.fit(X_train, y_train) #Fit the SVM model according to the given training data.
# predicting on the training data
predict_train= regr.predict(X_train)

#testing the model training accuracy 
print(r2_score(y_train, predict_train))
print(mean_squared_error(y_train, predict_train))

In [ ]:
#Predicting on the test data
pred= regr.predict(X_test)
##testing the models accuracy on the test data
print(r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))

We observe that, when the R2 value increases and RMS error decreases from the previous model, we get a better performing model. Therefore, Standard scaler is the best fit for our model which can explain 86.78% of the variance of the training dataset and 86.52% of the variance of the test dataset. The prediction accuracy will vary +-2.3 (root mean squared error of 5.5).

# Same way we can compare between different karels

In [ ]:
#Split your data set into training (80%) and test data (20%)
X_train, X_test, y_train, y_test = train_test_split(X1, Energy, test_size=0.2, random_state=0, shuffle= "False")
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

#building the regressor and fit the training data to the regressor
regr = SVR(kernel='poly', degree=5) # y = ax5 + bx4 + cx3 + dx2 + ex + f

# fitting the regression model to the training data
regr.fit(X_train, y_train) #Fit the SVM model according to the given training data.
# predicting on the training data
predict_train= regr.predict(X_train)

#testing the model training accuracy 
print(r2_score(y_train, predict_train))
print(mean_squared_error(y_train, predict_train))